# Virtual Calculator
##### Bị cách ly buồn quá nên làm chơi

# Cài các thư viện sau

In [1]:
pip install cvzone

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Using legacy 'setup.py install' for cvzone, since package 'wheel' is not installed.
    Running setup.py install for cvzone: started
    Running setup.py install for cvzone: finished with status 'done'



In [2]:
pip install mediapipe

Note: you may need to restart the kernel to use updated packages.


# Tiến Hành
* Import các thư viện sau, thư viện HandDetector sẽ xem các cử động tay

In [1]:
import cv2
from cvzone.HandTrackingModule import HandDetector

* Thiết Lập Webcam, HandDector

In [2]:
cap = cv2.VideoCapture(0)

# chỉnh độ rộng id = 3, độ cao id=4
cap.set(3,1280)
cap.set(4,720)

detector = HandDetector(detectionCon=0.8, maxHands=1)

* Tạo class cho các nút bấm máy tính

In [3]:
class Button:
    def __init__(self, pos, width, height, value):
        
        self.pos = pos
        self.width = width
        self.height = height
        self.value = value
        
    def draw(self, img):
        # Vẽ hình chữ nhật sử dụng như nút bấm máy tính
        # Fill màu
        cv2.rectangle(img, 
                      self.pos, 
                      (self.pos[0] + self.width, self.pos[1] + self.height),
                      (225,225,225),
                      cv2.FILLED)
        # vẽ border
        cv2.rectangle(img, 
                      self.pos, 
                      (self.pos[0] + self.width, self.pos[1] + self.height),
                      (50,50,50),
                      2)
        
        cv2.putText(img,
                    self.value, 
                    (self.pos[0]+20, self.pos[1]+40),
                    cv2.FONT_HERSHEY_PLAIN, 
                    2, 
                    (50, 50,50), 
                    2)
        
    def checkClick(self, x,y):
        if self.pos[0] < x < self.pos[0] + self.width and self.pos[1] < y < self.pos[1] + self.height:
            cv2.rectangle(img, 
                          self.pos, 
                          (self.pos[0] + self.width, self.pos[1] + self.height),
                          (255,255,255),
                          cv2.FILLED)
            # vẽ border
            cv2.rectangle(img, 
                          self.pos, 
                          (self.pos[0] + self.width, self.pos[1] + self.height),
                          (50,50,50),
                          2)

            cv2.putText(img,
                        self.value, 
                        (self.pos[0] + 15, self.pos[1] + 45),
                        cv2.FONT_HERSHEY_PLAIN, 
                        3, 
                        (0,0,0), 
                        3)
            return True
        else:
            return False
        
        
        

* Khởi tạo button

In [4]:
# tên của nút
buttonListValues = [['7', '8', '9', '*'],
                    ['4', '5', '6', '-'],
                    ['1', '2', '3', '+'],
                    ['0', '/', '.', '=']]

# Tạo nút
buttonList = []
for x in range (4):
    for y in range(4):
        xpos = x * 60 + 300
        ypos = y * 60 + 120
        buttonList.append( Button( (xpos,ypos), 60, 60, buttonListValues[y][x] ) )

In [5]:
myEquation = ''
delayCounter = 0

* Main Code

In [6]:
while True:
    # Lấy hình ảnh từ webcam, bắt chuyển động
    success, img = cap.read()
    
    # Tránh cho hình ảnh bị ngược, ta cần flip nó theo chiều dọc (1), ngang thì (0)
    img = cv2.flip(img,1)
    
    # Phân tích tay
    hands, img = detector.findHands(img, flipType=False)
    
    # Draw nút
    cv2.rectangle(img, 
                  (300, 60), 
                  (300 + 240, 60 + 60),
                  (225,225,225),
                  cv2.FILLED)

    cv2.rectangle(img, 
              (300, 60), 
              (300 + 240, 60 + 60 ),
              (50,50,50),
              2)
    
   
  
    for button in buttonList:
        button.draw(img)

    # Xét Tay
    if hands:
        lmList = hands[0]['lmList']
        
        #Xét khoảng cách 2 ngón tay
        length, _, img = detector.findDistance(lmList[8], lmList[12], img)
        #print(length)
        
        x,y = lmList[8]
        
        if length < 50:
            for i, button in enumerate(buttonList):
                
                # Tính Toán
                if button.checkClick(x,y) and delayCounter == 0:
                    myValues = buttonListValues[int(i%4)][int(i/4)]
                    if myValues == '=':
                        myEquation = str(eval(myEquation))
                    else:
                        myEquation += myValues
                    delayCounter = 1
                    
    # Tránh bị đè nút
    if delayCounter != 0:
        delayCounter += 1
        if delayCounter > 10:
            delayCounter = 0
        
    # Hiện kết quả
    cv2.putText(img,
                myEquation, 
                (300 + 10, 60 + 45),
                cv2.FONT_HERSHEY_PLAIN, 
                2, 
                (50, 50,50), 
                2)
    # Hiện hình ảnh
    cv2.imshow('Image', img)
    
    # Delay 1 milli second
    key = cv2.waitKey(1)
    
    # esc
    if key == 27:
        cv2.destroyAllWindows()
        myEquation = ''
        break
    
    if key == ord('c'):
        myEquation = ''